In [1]:
# Import thư viện Machine Learning và Data Science 
import tensorflow_probability as tfp
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import tensorflow_hub as hub
from skimage import exposure
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
import scipy

In [2]:
# Import các thư viện Built In
from datetime import datetime
from glob import glob
import warnings
import IPython
import urllib
import zipfile
import pickle
import shutil
import string
import math
import tqdm
import time
import os
import gc
import re

In [3]:
# Import các thư viện hỗ trợ Visualization 
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib
import plotly
import PIL
import cv2

In [4]:
# Các PRESET thông dụng
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", 15)]
LABEL_COLORS_WOUT_NO_FINDING = LABEL_COLORS[:8]+LABEL_COLORS[9:]

In [5]:
# Các Import khác
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm.notebook import tqdm
import pydicom

In [6]:
# Định nghĩa đường dẫn đến root data directory
DATA_DIR = "/kaggle/input/vinbigdata-chest-xray-abnormalities-detection"

In [7]:
# Đường dẫn đến training & testing dicom folders tương ứng
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [8]:
# Lấy đường dẫn của tất cả ảnh trong 2 folder trên đưa vào list
TRAIN_DICOM_PATHS = [os.path.join(TRAIN_DIR, f_name) for f_name in os.listdir(TRAIN_DIR)]
TEST_DICOM_PATHS = [os.path.join(TEST_DIR, f_name) for f_name in os.listdir(TEST_DIR)]
# Sau đó thống kê số lượng mỗi tập
print(f"\n... Số lượng ảnh trong tập train: {len(TRAIN_DICOM_PATHS)} ...")
print(f"... Số lượng ảnh trong tập test: {len(TEST_DICOM_PATHS)} ...")


... Số lượng ảnh trong tập train: 15000 ...
... Số lượng ảnh trong tập test: 3000 ...


In [9]:
# Định nghĩa paths đến các file .csv
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")

In [10]:
# Đọc các file .csv và tạo các dataframe object
train_df = pd.read_csv(TRAIN_CSV)
ss_df = pd.read_csv(SS_CSV)
# Sau đó xuất ra head(10) của các df
print("\n\nTRAIN DATAFRAME\n\n")
display(train_df.head(10))
print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
display(ss_df.head(10))



TRAIN DATAFRAME




,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
5,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0
7,47ed17dcb2cbeec15182ed335a8b5a9e,Nodule/Mass,8,R9,557.0,2352.0,675.0,2484.0
8,d3637a1935a905b3c326af31389cb846,Aortic enlargement,0,R10,1329.0,743.0,1521.0,958.0
9,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0




SAMPLE SUBMISSION DATAFRAME




,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,14 1 0 0 1 1
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,14 1 0 0 1 1
5,00b7e6bfa4dc1fe9ddd0ce74743e38c2,14 1 0 0 1 1
6,011295e0bcdc7636569ab73bfdcc4450,14 1 0 0 1 1
7,013c169f9dad6f1f6485da961b9f7bf2,14 1 0 0 1 1
8,01431a2618c0ace741e4e270a37e20b9,14 1 0 0 1 1
9,0168eb925aa6f28a78b16134792f5d0e,14 1 0 0 1 1


In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67914 entries, 0 to 67913
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   image_id    67914 non-null  object 
 1   class_name  67914 non-null  object 
 2   class_id    67914 non-null  int64  
 3   rad_id      67914 non-null  object 
 4   x_min       36096 non-null  float64
 5   y_min       36096 non-null  float64
 6   x_max       36096 non-null  float64
 7   y_max       36096 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 4.1+ MB


# **CÁC HÀM TỰ ĐỊNH NGHĨA**

In [12]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    """ Chuyển một file dicom thành một numpy array 
    
    Args:
        path (str): Đường dẫn đến dicom file cần convert
        voi_lut (bool): Dùng để chuyển dữ liệu DICOM raw sang chế độ xem "human-friendly"
        fix_monochrome (bool): Chưa mô tả
        
    Returns:
        Numpy array của dicom file tương ứng
        
    """
    # Dùng tvien pydicom để đọc dicom file
    dicom = pydicom.read_file(path)
    
    # Nếu có VOI LUT (trong trường hợp available bởi thiết bị DICOM)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
        
    # Đến đoạn này ảnh XRAY có vẻ bị inverted
    # Tìm thấy cách sửa là do monochrome
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    # Normalize image array này và return
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [13]:
def plot_image(img, title="", figsize=(8,8), cmap=None):
    """ Hàm plot image để tiết kiệm thời gian """
    plt.figure(figsize=figsize)
    
    if cmap:
        plt.imshow(img, cmap=cmap)
    else:
        img
        plt.imshow(img)
        
    plt.title(title, fontweight="bold")
    plt.axis(False)
    plt.show()

In [14]:
def get_image_id(path):
    """ Hàm trả về image-id từ path """
    return path.rsplit("/", 1)[1].rsplit(".", 1)[0]

In [15]:
def create_fractional_bbox_coordinates(row):
    """ Hàm trả về tọa độ bbox dưới dạng thập phân từ row của df """
    frac_x_min = row["x_min"]/row["img_width"]
    frac_x_max = row["x_max"]/row["img_width"]
    frac_y_min = row["y_min"]/row["img_height"]
    frac_y_max = row["y_max"]/row["img_height"]
    # Trả về định dạng [x_min, x_max, y_min, y_max]
    return frac_x_min, frac_x_max, frac_y_min, frac_y_max

In [16]:
def draw_bboxes(img, tl, br, rgb, label="", label_location="tl", opacity=0.1, line_thickness=0):
    """ Hàm vẽ các bbox lên ảnh """
    rect = np.uint8(np.ones((br[1]-tl[1], br[0]-tl[0], 3))*rgb)
    sub_combo = cv2.addWeighted(img[tl[1]:br[1],tl[0]:br[0],:], 1-opacity, rect, opacity, 1.0)    
    img[tl[1]:br[1],tl[0]:br[0],:] = sub_combo

    if line_thickness>0:
        img = cv2.rectangle(img, tuple(tl), tuple(br), rgb, line_thickness)
        
    if label:
        # MẶC ĐỊNH
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        FONT_SCALE = 1.666
        FONT_THICKNESS = 3
        FONT_LINE_TYPE = cv2.LINE_AA
        
        if type(label)==str:
            LABEL = label.upper().replace(" ", "_")
        else:
            LABEL = f"CLASS_{label:02}"
        
        text_width, text_height = cv2.getTextSize(LABEL, FONT, FONT_SCALE, FONT_THICKNESS)[0]
        
        label_origin = {"tl":tl, "br":br, "tr":(br[0],tl[1]), "bl":(tl[0],br[1])}[label_location]
        label_offset = {
            "tl":np.array([0, -10]), "br":np.array([-text_width, text_height+10]), 
            "tr":np.array([-text_width, -10]), "bl":np.array([0, text_height+10])
        }[label_location]
        img = cv2.putText(img, LABEL, tuple(label_origin+label_offset), 
                          FONT, FONT_SCALE, rgb, FONT_THICKNESS, FONT_LINE_TYPE)
    
    return img

# **THĂM DÒ CỘT IMAGE_ID**

**THỐNG KÊ TỔNG SỐ BBOX TRÊN MỖI BỨC ẢNH**

In [17]:
train_df.image_id

0        50a418190bc3fb1ef1633bf9678929b3
1        21a10246a5ec7af151081d0cd6d65dc9
2        9a5094b2563a1ef3ff50dc5c7ff71345
3        051132a778e61a86eb147c7c6f564dfe
4        063319de25ce7edb9b1c6b8881290140
                       ...               
67909    936fd5cff1c058d39817a08f58b72cae
67910    ca7e72954550eeb610fe22bf0244b7fa
67911    aa17d5312a0fb4a2939436abca7f9579
67912    4b56bc6d22b192f075f13231419dfcc8
67913    5e272e3adbdaafb07a7e84a9e62b1a4c
Name: image_id, Length: 67914, dtype: object

In [18]:
train_df.image_id.value_counts()

03e6ecfa6f6fb33dfeac6ca4f9b459c9    57
fa109c087e46fe1ea27e48ce6d154d2f    52
e31be972e181987a8600a8700c1ebe88    48
ecf474d5d4f65d7a3e23370a68b8c6a0    46
6d5acf3f8a973a26844d617fffe72998    46
                                    ..
f5258c6553ebe9557f5f79ba52428220     3
382145f051a1d66c85a36159c2e8a743     3
7af97a01d24c91deb8fe5355014be936     3
70ae292359ad785182a9cd214c26892f     3
f5f140184ce64f81502833e45d4c04ef     3
Name: image_id, Length: 15000, dtype: int64

In [19]:
fig = px.histogram(train_df.image_id.value_counts(), 
                   log_y=False, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Số lượng bbox trên mỗi bức ảnh"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX TRÊN MỖI BỆNH NHÂN</b>" \
                         "<i><sub>(Không Log Scale trục y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số bbox trên một bệnh nhân</b>",
                  yaxis_title="<b>Số lượng bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [20]:
fig = px.histogram(train_df.image_id.value_counts(), 
                   log_y=True, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Số lượng bbox trên mỗi bức ảnh"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX TRÊN MỖI BỆNH NHÂN</b>" \
                         "<i><sub>(Có Log Scale trục y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số bbox trên một bệnh nhân</b>",
                  yaxis_title="<b>Số lượng bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [21]:
fig = px.histogram(train_df.image_id.value_counts().loc[lambda x : x>3], 
                   log_y=False, color_discrete_sequence=['indianred'], opacity=0.7,
                   labels={"value":"Số lượng bbox trên mỗi bức ảnh"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX TRÊN MỖI BỆNH NHÂN</b>" \
                         "<i><sub>(Bỏ giá trị 3 và không Log Scale trục y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số bbox trên một bệnh nhân</b>",
                  yaxis_title="<b>Số lượng bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [22]:
scipy.stats.skew(train_df.image_id.value_counts().values)

3.8687405565463298

In [23]:
exp = train_df.image_id.value_counts().loc[lambda x : x>3]
scipy.stats.skew(exp)

2.524988166279376

**ĐẾM SỐ CĂN BỆNH TRÊN 1 BỨC ẢNH**

In [24]:
train_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
67909,936fd5cff1c058d39817a08f58b72cae,No finding,14,R1,NaN,NaN,NaN,NaN
67910,ca7e72954550eeb610fe22bf0244b7fa,No finding,14,R1,NaN,NaN,NaN,NaN
67911,aa17d5312a0fb4a2939436abca7f9579,No finding,14,R8,NaN,NaN,NaN,NaN
67912,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0


In [25]:
train_df.groupby('image_id')["class_id"].unique()

image_id
000434271f63a053c4128a0ba6352c7f                 [14]
00053190460d56c53cc3e57321387478                 [14]
0005e8e3701dfb1dd93d53e2ff537b6e         [7, 8, 6, 4]
0006e0a85696f6bb578e84fafa9a5607                 [14]
0007d316f756b3fa0baea2ff514ce945    [13, 11, 3, 0, 5]
                                          ...        
ffe6f9fe648a7ec29a50feb92d6c15a4            [3, 0, 9]
ffea246f04196af602c7dc123e5e48fc                 [14]
ffeffc54594debf3716d6fcd2402a99f                  [0]
fff0f82159f9083f3dd1f8967fc54f6a                 [14]
fff2025e3c1d6970a8a6ee0404ac6940                 [14]
Name: class_id, Length: 15000, dtype: object

In [26]:
train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(x))

image_id
000434271f63a053c4128a0ba6352c7f    1
00053190460d56c53cc3e57321387478    1
0005e8e3701dfb1dd93d53e2ff537b6e    4
0006e0a85696f6bb578e84fafa9a5607    1
0007d316f756b3fa0baea2ff514ce945    5
                                   ..
ffe6f9fe648a7ec29a50feb92d6c15a4    3
ffea246f04196af602c7dc123e5e48fc    1
ffeffc54594debf3716d6fcd2402a99f    1
fff0f82159f9083f3dd1f8967fc54f6a    1
fff2025e3c1d6970a8a6ee0404ac6940    1
Name: class_id, Length: 15000, dtype: int64

In [27]:
train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a : a != 14, x))))

image_id
000434271f63a053c4128a0ba6352c7f    0
00053190460d56c53cc3e57321387478    0
0005e8e3701dfb1dd93d53e2ff537b6e    4
0006e0a85696f6bb578e84fafa9a5607    0
0007d316f756b3fa0baea2ff514ce945    5
                                   ..
ffe6f9fe648a7ec29a50feb92d6c15a4    3
ffea246f04196af602c7dc123e5e48fc    0
ffeffc54594debf3716d6fcd2402a99f    1
fff0f82159f9083f3dd1f8967fc54f6a    0
fff2025e3c1d6970a8a6ee0404ac6940    0
Name: class_id, Length: 15000, dtype: int64

In [28]:
tmp_df = train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a: a != 14, x))))
# tmp_df.value_counts().sort_index()
fig = px.histogram(tmp_df, 
             log_y=False, color_discrete_sequence=['skyblue'], opacity=0.7,
             labels={"value":"Số lượng căn bệnh của họ"},
             title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG CĂN BỆNH CỦA BỆNH NHÂN   " \
                   "<i><sub>(KHÔNG Log Scale trục Y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số lượng căn bệnh của bệnh nhân</b>",
                  yaxis_title="<b>Số bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [29]:
tmp_df = train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a: a != 14, x))))
# tmp_df.value_counts().sort_index()
fig = px.histogram(tmp_df, 
             log_y=True, color_discrete_sequence=['skyblue'], opacity=0.7,
             labels={"value":"Số lượng căn bệnh của họ"},
             title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG CĂN BỆNH CỦA BỆNH NHÂN   " \
                   "<i><sub>(CÓ Log Scale trục Y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số lượng căn bệnh của bệnh nhân</b>",
                  yaxis_title="<b>Số bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

In [30]:
tmp_df = train_df.groupby('image_id')["class_id"].unique().apply(lambda x: len(list(filter(lambda a: a != 14, x)))).loc[lambda x: x!=0]
# tmp_df.value_counts().sort_index()
fig = px.histogram(tmp_df, 
             log_y=False, color_discrete_sequence=['skyblue'], opacity=0.7,
             labels={"value":"Số lượng căn bệnh của họ"},
             title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG CĂN BỆNH CỦA BỆNH NHÂN   " \
                   "<i><sub>(LOẠI BỎ CLASS 14, KHÔNG Log Scale trục Y)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Số lượng căn bệnh của bệnh nhân</b>",
                  yaxis_title="<b>Số bệnh nhân</b>",
                  font=FIG_FONT,)
fig.show()

# THĂM DÒ CỘT CLASS_NAME

**Đếm số bbox đối với mỗi class**

In [31]:
fig = px.bar(train_df.class_name.value_counts(), 
             color=train_df.class_name.value_counts().index, opacity=0.85,
             color_discrete_sequence=LABEL_COLORS_WOUT_NO_FINDING,
             labels={"y":"Số bbox", "x":""},
             title="<b>PHÂN PHỐI SỐ LƯỢNG BBOX THEO CLASS</b>",)
fig.update_layout(legend_title=None,
                  font=FIG_FONT,
                  xaxis_title="",
                  yaxis_title="<b>Số bbox trên mỗi class</b>")

fig.show()

# THĂM DÒ CỘT CLASS_ID

*Bước này không có gì để phân tích. Chủ yếu là convert class_name sang class_id*

Ngoài ra còn tạo các dict int_2_str, str_2_int,str_2_clr... 

In [32]:
# Create dictionary mappings
int_2_str = {i:train_df[train_df["class_id"]==i].iloc[0]["class_name"] for i in range(15)}
str_2_int = {v:k for k,v in int_2_str.items()}
int_2_clr = {str_2_int[k]:LABEL_COLORS[i] for i,k in enumerate(sorted(str_2_int.keys()))}

print("\n... Dictionary Mapping Class Integer to Class String Representation [int_2_str]...\n")
display(int_2_str)

print("\n... Dictionary Mapping Class String to Class Integer Representation [str_2_int]...\n")
display(str_2_int)

print("\n... Dictionary Mapping Class Integer to Color Representation [str_2_clr]...\n")
display(int_2_clr)

print("\n... Head of Train Dataframe After Dropping The Class Name Column...\n")
train_df.drop(columns=["class_name"], inplace=True)
display(train_df.head(5))


... Dictionary Mapping Class Integer to Class String Representation [int_2_str]...



{0: 'Aortic enlargement',
 1: 'Atelectasis',
 2: 'Calcification',
 3: 'Cardiomegaly',
 4: 'Consolidation',
 5: 'ILD',
 6: 'Infiltration',
 7: 'Lung Opacity',
 8: 'Nodule/Mass',
 9: 'Other lesion',
 10: 'Pleural effusion',
 11: 'Pleural thickening',
 12: 'Pneumothorax',
 13: 'Pulmonary fibrosis',
 14: 'No finding'}


... Dictionary Mapping Class String to Class Integer Representation [str_2_int]...



{'Aortic enlargement': 0,
 'Atelectasis': 1,
 'Calcification': 2,
 'Cardiomegaly': 3,
 'Consolidation': 4,
 'ILD': 5,
 'Infiltration': 6,
 'Lung Opacity': 7,
 'Nodule/Mass': 8,
 'Other lesion': 9,
 'Pleural effusion': 10,
 'Pleural thickening': 11,
 'Pneumothorax': 12,
 'Pulmonary fibrosis': 13,
 'No finding': 14}


... Dictionary Mapping Class Integer to Color Representation [str_2_clr]...



{0: 'rgb(193, 39, 74)',
 1: 'rgb(221, 74, 76)',
 2: 'rgb(240, 103, 68)',
 3: 'rgb(249, 142, 82)',
 4: 'rgb(253, 181, 103)',
 5: 'rgb(254, 212, 129)',
 6: 'rgb(254, 236, 159)',
 7: 'rgb(255, 255, 190)',
 14: 'rgb(239, 249, 166)',
 8: 'rgb(214, 238, 155)',
 9: 'rgb(177, 223, 163)',
 10: 'rgb(134, 207, 165)',
 11: 'rgb(94, 185, 169)',
 12: 'rgb(61, 149, 184)',
 13: 'rgb(68, 113, 178)'}


... Head of Train Dataframe After Dropping The Class Name Column...



,image_id,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,14,R10,NaN,NaN,NaN,NaN


# THĂM DÒ CỘT RAD_ID 

In [35]:
fig = px.histogram(train_df, x="rad_id", color="rad_id",opacity=0.85,
                   labels={"rad_id":"Radiologist ID"},
                   title="<b>PHÂN PHỐI CỦA SỐ LƯỢNG BBOX ĐƯỢC TẠO RA CỦA MỖI BÁC SĨ</b>",
                   ).update_xaxes(categoryorder="total descending")
fig.update_layout(legend_title="<b>RADIOLOGIST ID</b>",
                  xaxis_title="<b>Radiologist ID</b>",
                  yaxis_title="<b>Số lượng bbox được tạo ra</b>",
                  font=FIG_FONT,)
fig.show()

# Một số code thử nghiệm và code lỗi

In [34]:
train_df.class_name.value_counts().drop("No finding")

AttributeError: 'DataFrame' object has no attribute 'class_name'

In [36]:
fig = px.bar(train_df.class_name.value_counts(), 
             color=train_df.class_name.value_counts().index, opacity=0.85,
             color_discrete_sequence=LABEL_COLORS_WOUT_NO_FINDING,
             labels={"y":"Annotations Per Class", "x":""},
             title="<b>Annotations Per Class</b>",)
fig.update_layout(legend_title=None,
                  font=FIG_FONT,
                  xaxis_title="",
                  yaxis_title="<b>Annotations Per Class</b>")

fig.show()

AttributeError: 'DataFrame' object has no attribute 'class_name'

In [37]:
fig = px.histogram(train_df.groupby('image_id')["class_name"].unique().apply(lambda x: len(x)), 
             log_y=False, color_discrete_sequence=['skyblue'], opacity=0.8,
             labels={"value":"Number of Unique Abnormalities"},
             title="<b>DISTRIBUTION OF # OF ANNOTATIONS PER PATIENT   " \
                   "<i><sub>(Log Scale for Y-Axis)</sub></i></b>",
                   )
fig.update_layout(showlegend=False,
                  xaxis_title="<b>Number of Unique Abnormalities</b>",
                  yaxis_title="<b>Count of Unique Patients</b>",
                  font=FIG_FONT,)
fig.show()

KeyError: 'Column not found: class_name'

.update_xaxes(categoryorder="total descending")

Cái này giúp sort